In [1]:
import pathlib
from Utils.wetland_selection    import get_top_scenes
from Utils.treatment_regression import logistic_process
from Utils.outcome_syndata_generator import generate_synthetic_outcome
from Utils.treatment_syndata_generator import generate_synthetic_treatment
from Utils.outcome_regression import regression_process
import numpy as np

In [2]:
# top scenes by wetland development
top_df = get_top_scenes(num_scenes=5)
scene_ids = top_df['scene'].astype(str).tolist()

# Step 2: Define folder paths for rasters
DATA_DIR = pathlib.Path('/Users/sayedmorteza/Library/CloudStorage/Box-Box/Hetwet_Data')         

# folders
folders = {
    'wet': DATA_DIR / 'WETLAND_DEV_1996_2016',
    'dem': DATA_DIR / 'DEM',
    'cap': DATA_DIR / 'CAPITAL_1996',
    'claims_96': DATA_DIR / 'LOG_CLAIMS_1996',
    'claims_16': DATA_DIR / 'LOG_CLAIMS_2016',

}

In [3]:
# Step 3: Run logistic regression on selected scenes
theta = 0.5  # classification threshold
logit_res = logistic_process(
    scene_ids=scene_ids,
    folders=folders,
    threshold=theta,
    regularization_C=1.0,
    noise_sd=0.1,
    noise_type='gaussian'
)

INFO: Processing scene 3043
INFO: Processing scene 2985
INFO: Processing scene 3095
INFO: Processing scene 2779
INFO: Processing scene 2922
INFO: Coefficients saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/treatment_logistic_coeffs.txt
INFO: Maps saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/scene_3043_treatment_maps.pdf


In [4]:

generate_synthetic_treatment(
    scene_ids=scene_ids,
    folders=folders,
    logit_pipe=logit_res,
    threshold=theta,
    noise_sd=0.1,
    noise_type='gaussian'
)

print("Completed: logistic model fitted and synthetic wetland maps generated.")


INFO: [gaussian] Saved synthetic for 3043
INFO: [gaussian] Saved synthetic for 2985
INFO: [gaussian] Saved synthetic for 3095
INFO: [gaussian] Saved synthetic for 2779
INFO: [gaussian] Saved synthetic for 2922
INFO: All outputs written to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results


Completed: logistic model fitted and synthetic wetland maps generated.


In [5]:
outcome_reg = regression_process(
    scene_ids=scene_ids,
    folders=folders,
    noise_sd=0.1,
    noise_type='gaussian'
)

INFO: Loading scene 3043
INFO: Loading scene 2985
INFO: Loading scene 3095
INFO: Loading scene 2779
INFO: Loading scene 2922
INFO: Coefficients saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/outcome_regression_coeffs.txt
INFO: Maps saved to /Users/sayedmorteza/Library/CloudStorage/Box-Box/Caltech Research/Scripts/ImageTRT/Synthetic Data Generation/Results/scene_3043_continuous_outcome.pdf


In [7]:
# then generate synthetic outcomes:
generate_synthetic_outcome(
    scene_ids = scene_ids,
    folders = folders,
    reg_pipe = outcome_reg,
    target_shape = (8, 8),
    noise_type = "gaussian",
    noise_sd = 0.2,
    results_dir = None
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()